# BERT Document Classification Tutorial
> BERT Document Classification Tutorial  with Code
> 
> ref: https://www.youtube.com/watch?v=_eSGWNqKeeY

- toc: true
- branch: master
- badges: true
- comments: true
- categories: [bert, jupyter]
- hide: false
- search_exclude: true

# Classifying Wikipedia Comments with BERT

# Contents

# Part I - Setup & Dataset Prep

## 1. Setup

## 2. Retrieve & Inspect Dataset

## 3. BERT Input Length Limitation

# Part II - BERT Fine-Tuning

## 4. Train Our Classification Model

## 5. Performance On Test Set

## 6. Save Trained Model

# Part III - Semantic Similarity

## 7. Vectorize Comments

## 8. Semantic Similarity Search